In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler

#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/DontGetKicked/training.csv', encoding = "ISO-8859-1", engine='python')
test = pd.read_csv('/kaggle/input/DontGetKicked/test.csv', encoding = "ISO-8859-1", engine='python')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
y = train['IsBadBuy']
X = train.drop(['IsBadBuy','RefId','PurchDate'], axis = 1)

In [ ]:
y

In [ ]:
X

In [ ]:
numerical_features = [c for c, dtype in zip(X.columns, X.dtypes)
                     if dtype.kind in ['i','f']]
categorical_features = [c for c, dtype in zip(X.columns, X.dtypes)
                     if dtype.kind not in ['i','f']]

In [ ]:
#import train_test_split library
from sklearn.model_selection import train_test_split

# create train test split
X_train, X_test, y_train, y_test = train_test_split( X,  y, test_size=0.3, random_state=0, stratify = y)

In [ ]:
preprocessor = make_column_transformer(

        (make_pipeline(
        SimpleImputer(strategy = 'median'),
        StandardScaler()), numerical_features),

        (make_pipeline(
        SimpleImputer(strategy = 'constant', fill_value = 'missing'),
        OneHotEncoder(categories = 'auto', handle_unknown = 'ignore')), categorical_features),

)

In [ ]:
Model = make_pipeline(preprocessor, RandomForestClassifier())

In [ ]:
Model.fit(X_train, y_train)

In [ ]:
print(f'Train : {Model.score(X_train, y_train):.3f}')
print(f'Test : {Model.score(X_test, y_test):.3f}')

In [ ]:
sub_test = test.drop(['RefId'], axis = 1)
sub_test_pred = Model.predict(sub_test).astype(int)
AllSub = pd.DataFrame({ 'RefId': test['RefId'],
                       'IsBadBuy' : sub_test_pred
    
})

AllSub.to_csv("DGK_Pipeline_RF.csv", index = False)